#### Génération du graph "communes" dans Neo4J

Notebook amorcé par : Jesshuan

Objectifs :

- Récupération du graph communes avec points d'intérêt touristiques sous forme edges et nodes depuis les notebooks de mika... (fichier finalement transféré à l'a'ncienne)

- jointure avec infos complémentaires sur les communes (nom, etc...)

- jointure avec infos complémentaires sur les immatriculations de véhicules

- Génération dans une base de données locale Neo4j

Imports :

In [1]:
import geopandas as gpd
import pandas as pd

from neo4j import GraphDatabase, basic_auth

In [2]:
PASSWORD_NEO4J = 'passwordneo4j'

--------

Récupération du fichier des neouds communes de Micka

In [3]:
data_path = "./../data_communes/[france]Gnodes_communes.feather"

In [4]:
nodes = gpd.read_feather(data_path).reset_index()

Cleaning :

In [5]:
nodes.drop(['area', 'geometry'], axis=1, inplace=True)

Jointure with others features for communes :

In [6]:
communes_infos = pd.read_feather('./../data_communes/communes.feather')

In [7]:
communes_infos

,insee,nom_commune,surf_ha,x,y
0,2B222,Pie-d'Orezza,573.0,9.338151,42.374290
1,2B137,Lano,824.0,9.235358,42.378872
2,2B051,Cambia,833.0,9.302108,42.368750
3,2B106,Érone,393.0,9.266614,42.375565
4,2B185,Oletta,2674.0,9.333845,42.641773
...,...,...,...,...,...
34950,62885,Westrehem,299.0,2.343461,50.544903
34951,62795,Sibiville,738.0,2.331172,50.304585
34952,62187,Buneville,384.0,2.347346,50.324764
34953,62694,Rebreuve-sur-Canche,842.0,2.340987,50.266950


Ici, on prend les informations de géolocalisations comme le centre "géométrique" de la communes (ces coordonnées vont devenir les coordonnées "x_centroid" et "y_centroid"). En effet, les coordonnées en x et y utilisés par la suite seront celles récupérées par Micka depuis OSM, qui sont les "vraies" coordonnées des centre-villes et centre-villages.

In [8]:
communes_infos.rename(columns={'x':'x_centroid','y':'y_centroid'}, inplace=True)

Jointure à gauche (pour enlever ce qui n'est pas métropolitain) :

In [9]:
df_join = nodes.merge(communes_infos, how='left', left_on='codgeo', right_on='insee')

Dernier éléments, les nombres d'immatriculations à jour :

In [10]:
immat = pd.read_feather('./../data_communes/immat.feather')

In [11]:
immat.rename(columns={'x':'x_centroid','y':'y_centroid'}, inplace=True)

Deuxième jointure :

In [12]:
df_join_tot = df_join.merge(immat, how='left', left_on='insee', right_on='codgeo')

In [13]:
df_join_tot.head()

,osmid,y,x,codgeo_x,sleeping,visit,working,fooding,vehicule,insee_x,...,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,insee_y,nom,surf_ha_y,x_centroid_y,y_centroid_y
0,1149451,50.582893,2.143174,62053,3.0,1.0,NaN,NaN,NaN,62053,...,200069037,CA du Pays de Saint-Omer,7.0,0.0,787.0,62053,Audincthun,1520.0,2.127048,50.578743
1,910704,49.717236,3.012390,80284,NaN,1.0,NaN,NaN,NaN,80284,...,200070985,CC de l'Est de la Somme,10.0,0.0,931.0,80284,Esmery-Hallon,1888.0,3.024816,49.711567
2,1452239,42.876728,0.610023,31290,NaN,NaN,NaN,NaN,NaN,31290,...,200072635,CC PyrÃ©nÃ©es Haut Garonnaises,1.0,0.0,43.0,31290,Lège,289.0,0.598100,42.879140
3,2173628,48.094594,1.514000,28400,NaN,NaN,NaN,NaN,NaN,28400,...,200070159,CC CÅur de Beauce,2.0,0.0,228.0,28400,Varize,1387.0,1.525235,48.082363
4,106436,47.193507,0.099943,37220,NaN,2.0,NaN,NaN,NaN,37220,...,200043081,"CC Chinon, Vienne et Loire",3.0,0.0,408.0,37220,Saint-Germain-sur-Vienne,1338.0,0.110709,47.180550


On clean les colonnes en trop :

In [14]:
columns_to_drop = ['insee_y','codgeo_y', 'codgeo_x', 'surf_ha_y', 'x_centroid_y', 'y_centroid_y']

In [15]:
df_join_tot.drop(columns_to_drop, axis=1, inplace=True)

In [16]:
df_join_tot.head()

,osmid,y,x,sleeping,visit,working,fooding,vehicule,insee_x,nom_commune,surf_ha_x,x_centroid_x,y_centroid_x,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,nom
0,1149451,50.582893,2.143174,3.0,1.0,NaN,NaN,NaN,62053,Audincthun,1520.0,2.127048,50.578743,200069037,CA du Pays de Saint-Omer,7.0,0.0,787.0,Audincthun
1,910704,49.717236,3.012390,NaN,1.0,NaN,NaN,NaN,80284,Esmery-Hallon,1888.0,3.024816,49.711567,200070985,CC de l'Est de la Somme,10.0,0.0,931.0,Esmery-Hallon
2,1452239,42.876728,0.610023,NaN,NaN,NaN,NaN,NaN,31290,Lège,289.0,0.598100,42.879140,200072635,CC PyrÃ©nÃ©es Haut Garonnaises,1.0,0.0,43.0,Lège
3,2173628,48.094594,1.514000,NaN,NaN,NaN,NaN,NaN,28400,Varize,1387.0,1.525235,48.082363,200070159,CC CÅur de Beauce,2.0,0.0,228.0,Varize
4,106436,47.193507,0.099943,NaN,2.0,NaN,NaN,NaN,37220,Saint-Germain-sur-Vienne,1338.0,0.110709,47.180550,200043081,"CC Chinon, Vienne et Loire",3.0,0.0,408.0,Saint-Germain-sur-Vienne


Après vérification, la colonne "nom" est moins remplie (de 5 lignes) que la colonne "nom_commune"...

In [17]:
df_join_tot.loc[df_join_tot['nom_commune']!=df_join_tot['nom'],:]

,osmid,y,x,sleeping,visit,working,fooding,vehicule,insee_x,nom_commune,surf_ha_x,x_centroid_x,y_centroid_x,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,nom
1802,2116433,49.236510,5.467500,NaN,1.0,NaN,NaN,NaN,55050,Bezonvaux,926.0,5.471458,49.231598,NaN,NaN,NaN,NaN,NaN,NaN
11433,2116434,49.238106,5.397626,NaN,1.0,NaN,NaN,NaN,55307,Louvemont-Côte-du-Poivre,831.0,5.401587,49.240210,NaN,NaN,NaN,NaN,NaN,NaN
19057,2967703,49.233925,5.265244,NaN,1.0,NaN,NaN,NaN,55139,Cumières-le-Mort-Homme,611.0,5.268401,49.234490,NaN,NaN,NaN,NaN,NaN,NaN
23990,280440,49.181231,-0.221640,NaN,NaN,NaN,NaN,NaN,14666,Sannerville,520.0,-0.224219,49.183563,NaN,NaN,NaN,NaN,NaN,NaN
25503,6837719,49.213400,1.330809,2.0,1.0,NaN,NaN,NaN,27058,Les Trois Lacs,3657.0,1.333144,49.216904,NaN,NaN,NaN,NaN,NaN,NaN


Donc, supression de cette colonne "nom" également :

In [18]:
df_join_tot.drop(['nom'], axis=1, inplace=True)

In [19]:
df_join_tot.head()

,osmid,y,x,sleeping,visit,working,fooding,vehicule,insee_x,nom_commune,surf_ha_x,x_centroid_x,y_centroid_x,epci,libepci,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp
0,1149451,50.582893,2.143174,3.0,1.0,NaN,NaN,NaN,62053,Audincthun,1520.0,2.127048,50.578743,200069037,CA du Pays de Saint-Omer,7.0,0.0,787.0
1,910704,49.717236,3.012390,NaN,1.0,NaN,NaN,NaN,80284,Esmery-Hallon,1888.0,3.024816,49.711567,200070985,CC de l'Est de la Somme,10.0,0.0,931.0
2,1452239,42.876728,0.610023,NaN,NaN,NaN,NaN,NaN,31290,Lège,289.0,0.598100,42.879140,200072635,CC PyrÃ©nÃ©es Haut Garonnaises,1.0,0.0,43.0
3,2173628,48.094594,1.514000,NaN,NaN,NaN,NaN,NaN,28400,Varize,1387.0,1.525235,48.082363,200070159,CC CÅur de Beauce,2.0,0.0,228.0
4,106436,47.193507,0.099943,NaN,2.0,NaN,NaN,NaN,37220,Saint-Germain-sur-Vienne,1338.0,0.110709,47.180550,200043081,"CC Chinon, Vienne et Loire",3.0,0.0,408.0


On renomme pour effacer les "_x" qui ont été générés pendant les jointures :

In [20]:
df_join_tot.rename(columns={'insee_x':'insee', 'surf_ha_x':'surf_ha','x_centroid_x':'x_centroid', 'y_centroid_x':'y_centroid'}, inplace=True)

In [21]:
df_join_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34460 entries, 0 to 34459
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   osmid                    34460 non-null  object 
 1   y                        34460 non-null  float64
 2   x                        34460 non-null  float64
 3   sleeping                 15412 non-null  float64
 4   visit                    17921 non-null  float64
 5   working                  6950 non-null   float64
 6   fooding                  6616 non-null   float64
 7   vehicule                 4166 non-null   float64
 8   insee                    34460 non-null  object 
 9   nom_commune              34460 non-null  object 
 10  surf_ha                  34460 non-null  float64
 11  x_centroid               34460 non-null  float64
 12  y_centroid               34460 non-null  float64
 13  epci                     34455 non-null  object 
 14  libepci               

--------------------------

GENERATION SOUS NEO4J

Fonctions de requêtes neo4j :

---

ATTENTION requêtes de suppressions (fonction d'ardoise !...)

In [38]:
''' def delete_all(tx):
    query = 'MATCH (n) \
                DETACH DELETE n \
                RETURN count(*) AS COUNT'
    result = tx.run(query)
    return result.data() '''

In [20]:
""" def delete_city(tx):
    
    query = "call apoc.periodic.iterate('MATCH (c:CITY) RETURN c','DETACH DELETE c RETURN count(*)', {batchSize:10000}) yield batches, total return batches, total"

    result = tx.run(query)
    return result.data() """


In [16]:
""" def delete_city(tx):
    query = 'MATCH (c:CITY) \
                CALL { WITH c \
                DETACH DELETE c \
                } IN TRANSACTIONS OF 10000 ROWS'
    result = tx.run(query)
    return result.data() """

In [56]:
''' def delete_just_edges(tx):
    query = 'MATCH ()-[r]-() \
                DELETE r \
                RETURN count(*) AS COUNT'
    result = tx.run(query)
    return result.data() '''

---

Requête de génération des noeuds communes :

In [23]:
def create_node_per_batch(tx, props_list):

    query = "   UNWIND $props_list AS map \
                CREATE (n:CITY) \
                SET n = map \
                RETURN count(n) AS COUNT"

    result = tx.run(query, props_list=props_list)
    return result.data()

---

Drivers session for Neo4j :

In [24]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", PASSWORD_NEO4J))

---

Attention ! LAncement des re^quetes "ardoises" (pour effacer le graph ou des éléments du graph en cas d'erreur)

Si besoin, dé-commenter les requêtes plus hauts, et les requêtes effectives ci-dessous

Ardoise (pour effacer tous les points cities)

In [21]:
""" with driver.session() as session:

    count_list = []

    result = session.execute_write(delete_city)

driver.close() """


Ardoise (pour effacer tout ancien graph)

In [ ]:
''' with driver.session() as session:

    count_list = []

    result = session.execute_write(delete_all)

driver.close()

nb_nodes = result[0]['COUNT']
                
print(f'Total Numbers of deleted nodes: {nb_nodes}') '''

Ardoise (pour effacer seulement les relations)

In [158]:
''' with driver.session() as session:

    count_list = []

    result = session.execute_write(delete_just_edges)

driver.close()

nb_edges = result[0]['COUNT']
                
print(f'Total Numbers of deleted edges: {nb_edges}') '''

Total Numbers of deleted edges: 0


---

NODES GENERATION :

In [25]:
print("conversion of objects to parameters list for neo4j...")

props_list = df_join_tot.to_dict("records")

conversion of objects to parameters list for neo4j...


In [26]:
BATCH_SIZE = 10000

In [27]:
# --- Batch function ---

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]


In [28]:
with driver.session() as session:

    count_list = []

    # Batch the writing session :

    for props_batch in batch(props_list, BATCH_SIZE):
        result = session.write_transaction(create_node_per_batch, props_batch)
        count_list.append(result[0]['COUNT'])
        nb_updates = sum(count_list)
        print(f'Number of nodes created : {nb_updates}...')

        

    driver.close()

print(f'Bath_list: {count_list}')

print(f'Total Numbers of nodes created/updates: {nb_updates}')

/tmp/ipykernel_25075/3878216737.py:8: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(create_node_per_batch, props_batch)


Number of nodes created : 10000...
Number of nodes created : 20000...
Number of nodes created : 30000...
Number of nodes created : 34460...
Bath_list: [10000, 10000, 10000, 4460]
Total Numbers of nodes created/updates: 34460


---